In [1]:
# We are trying to classify using KNN and convolution operation

In [2]:
import numpy as np
import pandas as pd
import torch
import sklearn as sk
import glob
import PIL
import cv2
from sklearn.model_selection import train_test_split
import torchvision
# from torchsummary import summary
import torch.nn as nn
from tqdm import tqdm
import torch.utils.data as data_utils
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm

In [3]:
images = glob.glob('../../cats_dogs/train/*.jpg')

In [4]:
# 1 = DOG
# 0 = CAT
labels = [1 if x == 'DOG' else 0 for x in images]

In [5]:
def split_datasets(data, split_size = 0.05):
    train, val = train_test_split(data, test_size = split_size)
    return train, val

In [6]:
dataset = list(zip(images, labels))

In [7]:
train, val = split_datasets(dataset)

In [8]:
incep_model = torchvision.models.Inception3()

/home/paperspace/.local/lib/python3.9/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = incep_model.to(device)

In [10]:
model.fc = nn.Linear(2048, 10)

In [11]:
def read_images(img_list, img_shape):
    img_holder = np.zeros((len(img_list), img_shape[0], img_shape[1], 3))
    for x in tqdm(range(len(img_list))):
        img = PIL.Image.open(img_list[x])
        img = img.resize((img_shape[0], img_shape[1]))
        img_holder[x] = img
    img_holder = torch.from_numpy(img_holder)
    img_holder = img_holder.reshape((len(img_list), 3, img_shape[0], img_shape[1]))
    return img_holder

In [12]:
arr_img = read_images([x[0] for x in train], (86, 86))

100%|██████████| 23750/23750 [00:48<00:00, 492.47it/s]


In [13]:
incep_model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [14]:
incep_model = incep_model.to(device)

In [15]:
train_tensor = data_utils.TensorDataset(arr_img)
train_tensor = data_utils.DataLoader(train_tensor, batch_size=2)

In [16]:
preds_holder = torch.zeros(len(train) // 2, 2, 10)

In [17]:
# arr_img.shape

In [21]:
count = 0
for features in tqdm(train_tensor):
    features = features[0].float()
    features = features.to(device)
    preds = incep_model(features)
    preds_holder[count] = preds
    count += 1

 24%|██▎       | 2811/11875 [00:41<02:13, 68.03it/s]


KeyboardInterrupt: 

In [24]:
classifier = KNeighborsClassifier(n_neighbors=2)

In [34]:
preds_holder.shape

torch.Size([1485, 2, 10])

In [35]:
1485 * 2

2970

In [36]:
len(train)

23750

In [33]:
preds_holder.reshape().shape

torch.Size([29700])

In [ ]:
classifier.fit()